### Installation

In [5]:
! pip install --quiet "pytorch-lightning"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.8/722.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 60.2 MB/s eta 0:00:00


### Import dependencies


In [6]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
import torchvision.datasets
#from pytorch_lightning.metrics import functional
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import FashionMNIST
import matplotlib.pyplot as plt
import numpy as np

### Reproducibility

In [11]:
pl.seed_everything(42)

INFO:lightning_fabric.utilities.seed:Global seed set to 42


100%|██████████| 26421880/26421880 [00:01<00:00, 19387212.43it/s]


Extracting FashionMNIST/raw/train-images-idx3-ubyte.gz to FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 330505.72it/s]


Extracting FashionMNIST/raw/train-labels-idx1-ubyte.gz to FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6220368.62it/s]


Extracting FashionMNIST/raw/t10k-images-idx3-ubyte.gz to FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 34995586.70it/s]

Extracting FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to FashionMNIST/raw

